# 04 — Bayesian Logistic Regression (with Uncertainty)

This notebook fits a **Bayesian logistic regression** to predict ore-occurrence likelihood on a geospatial grid and **quantifies uncertainty**.

**Why these steps matter**
- **Drop constant/near-constant columns**: avoids singular directions that break the sampler on sparse, one-hot features.
- **Standardize features**: keeps coefficients on comparable scales (especially with mixed one-hot + continuous features like gravity).
- **Intercept prior = base rate**: set the prior mean for the intercept to `logit(prevalence)` so the model starts near realistic class odds in class-imbalanced data.
- **`logit_p=` in Bernoulli**: avoids numerical overflow of `sigmoid` at extreme logits.
- **Robust NUTS init (`jitter+adapt_diag`) + higher target_accept**: safer step-size and mass-matrix adaptation on real-world, imbalanced datasets.

**Outputs**
- `data/processed/mean_probs.npy` — posterior mean probability per grid cell  
- `data/processed/std_probs.npy` — posterior std-dev (uncertainty) per grid cell

These files power the Streamlit viewer and figure exporters.


In [1]:
## 1) Install deps
%pip install -q -r ../requirements.txt


Note: you may need to restart the kernel to use updated packages.


In [2]:
## 2) Imports & data loading
import os
import numpy as np
import joblib
import matplotlib.pyplot as plt

import pymc as pm
import arviz as az
from scipy.special import logit
from sklearn.metrics import roc_auc_score

from pathlib import Path

project_root = Path.cwd().parent

# Required inputs
Xc = np.load(project_root / "data/processed/X_coords.npy")       # (n, 2) or similar
y  = np.load(project_root / "data/processed/y_labels.npy")       # (n,)
grid = joblib.load(project_root / "data/processed/grid_gdf.joblib")  # GeoDataFrame of grid cells

# Optional inputs
parts = [Xc]

# Geology one-hots / encodings (optional)
geo_path = project_root / "data/processed/X_geo.npy"
if os.path.exists(geo_path):
    Xg = np.load(geo_path)
    if Xg.shape[0] == Xc.shape[0]:
        parts.append(Xg)
        print(f"Including geology features: {Xg.shape[1]} columns")
    else:
        print("Found X_geo.npy but row count mismatches X_coords; skipping geology.")

# Gravity (optional, continuous)
grav_path = project_root / "data/processed/X_gravity.npy"
if os.path.exists(grav_path):
    Xgrav = np.load(grav_path).reshape(-1, 1)
    if Xgrav.shape[0] == Xc.shape[0]:
        parts.append(Xgrav)
        print("Including gravity feature")
    else:
        print("Found X_gravity.npy but row count mismatches X_coords; skipping gravity.")

# Optional gravity gradient
grad_path = project_root / 'data/processed/X_gravity_grad.npy'
if os.path.exists(grad_path):
    Xgg = np.load(grad_path).reshape(-1, 1)
    if Xgg.shape[0] == Xc.shape[0]:
        parts.append(Xgg)
        print("Including gravity gradient")
        
# Optional geochemistry
geochem_path = project_root / "data/processed/X_geochem.npy"
if os.path.exists(geochem_path):
    Xgc = np.load(geochem_path)
    if Xgc.shape[0] == Xc.shape[0]:
        parts.append(Xgc)
        print(f"Including geochem: {Xgc.shape[1]} columns")
        
# Optional magnetic
if os.path.exists(project_root / "data/processed/X_mag.npy"):
    Xmag = np.load(project_root / "data/processed/X_mag.npy")
    if Xmag.shape[0] == Xc.shape[0]:   # or Xc.shape[0] if you build baseline first
        parts.append(Xmag)
        print(f"Including magnetics: {Xmag.shape[1]} cols")
        
# Final feature matrix
X = np.hstack(parts)
print("Feature matrix:", X.shape, "| Labels:", y.shape)

# Sanity checks
assert X.shape[0] == y.shape[0], "Row count mismatch between X and y"
assert set(np.unique(y)).issubset({0,1}), "y must be binary {0,1}"


Including geology features: 18 columns
Including gravity feature
Including gravity gradient
Including geochem: 8 columns
Including magnetics: 3 cols
Feature matrix: (7646, 33) | Labels: (7646,)


In [3]:
## 3) Preprocess for stability (drop constants, standardize)
# Remove constant / near-constant columns (important with one-hots)
std = X.std(axis=0, ddof=0)
keep = std > 1e-8
removed = int((~keep).sum())
if removed:
    print(f"Removed {removed} near-constant columns for stability")

Xb = X[:, keep]

# Standardize all remaining columns
mu = Xb.mean(axis=0)
sd = Xb.std(axis=0, ddof=0)
Xb = (Xb - mu) / sd

# Intercept prior at base rate (class prevalence)
p0 = float(np.clip(y.mean(), 1e-4, 1-1e-4))
mu_intercept = logit(p0)
print(f"Class prevalence p0={p0:.4f} -> intercept prior mean logit(p0)={mu_intercept:.3f}")


Removed 2 near-constant columns for stability
Class prevalence p0=0.0871 -> intercept prior mean logit(p0)=-2.350


In [4]:
## 4) Fit Bayesian logistic regression (PyMC, robust settings)
with pm.Model() as model:
    # Coefficients: moderately tight prior stabilizes with mixed feature scales
    beta = pm.Normal("beta", mu=0.0, sigma=0.5, shape=Xb.shape[1])
    # Intercept centered at base rate
    intercept = pm.Normal("intercept", mu=mu_intercept, sigma=2.0)

    logits = intercept + pm.math.dot(Xb, beta)

    # Use logit_p to avoid sigmoid overflow/underflow
    y_obs = pm.Bernoulli("y_obs", logit_p=logits, observed=y)

    trace = pm.sample(
        draws=1000,
        tune=1500,
        chains=4,
        cores=4,
        init="jitter+adapt_diag",
        target_accept=0.95,
        random_seed=42,
        progressbar=True,
    )


Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta, intercept]


Output()

Sampling 4 chains for 1_500 tune and 1_000 draw iterations (6_000 + 4_000 draws total) took 155 seconds.


In [5]:
## 5) Posterior predictive probabilities (mean & uncertainty) and AUC
# Extract posterior samples
beta_s = trace.posterior["beta"].values        # (chains, draws, n_features)
int_s  = trace.posterior["intercept"].values   # (chains, draws)

# logits: (chains, draws, n_obs) = (c,d,f) · (f,n) + intercept
logits = int_s[..., None] + np.tensordot(beta_s, Xb.T, axes=([2],[0]))
post_p = 1.0 / (1.0 + np.exp(-logits))  # sigmoid

mean_probs = post_p.mean(axis=(0,1))  # (n_obs,)
std_probs  = post_p.std(axis=(0,1))   # (n_obs,)

# Quick diagnostic
auc_bayes = roc_auc_score(y, mean_probs)
print(f"Bayesian ROC AUC: {auc_bayes:.3f}")


Bayesian ROC AUC: 0.680


In [6]:
## 6) Save arrays for app/figures
os.makedirs(project_root / "data/processed", exist_ok=True)
np.save(project_root / "data/processed/mean_probs.npy", mean_probs)
np.save(project_root / "data/processed/std_probs.npy",  std_probs)
print("Saved data/processed/mean_probs.npy and std_probs.npy")


Saved data/processed/mean_probs.npy and std_probs.npy
